In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from openai import AzureOpenAI
import os 
import requests
from bs4 import BeautifulSoup
import re 

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

In [ ]:
bing_grounded_sites = [
    'sec.gov',
    'finra.org/rules-guidance/rulebooks',
]

def bing_search(query):
    sites_str= ' OR '.join([f'site:{s}' for s in bing_grounded_sites])
    url = f'v7.0/search?q={query} ({sites_str})'
    print(f'Searching Bing: {url}')
    response = requests.get(f'{os.getenv("BING_ENDPOINT")}{url}', headers={'Ocp-Apim-Subscription-Key': os.getenv("BING_API_KEY")})
    return response

In [14]:
def get_result_clean_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for el in soup(['script', 'style', 'header', 'footer']):
        el.decompose()

    for el in soup.find_all(class_='sidebar'):
        el.decompose()

    text_content = soup.get_text()
    text_content = re.sub(r' {2,}', ' ', text_content)
    text_content = re.sub(r'\n{3,}', '\n\n', text_content)

    print(f'Extracted {len(text_content)} characters from {url}')

    return text_content



In [ ]:
def search_and_get_text(query):
    context = []
    response = bing_search(query)
    data = response.json()
    id = 1
    if 'webPages' in data:
        for page in data['webPages']['value']:
            url = page['url']
            text = get_result_clean_text(url)
            context.append({'id':id, 'src':url, 'content':text})
            id+=1
    
    # serialize answer
    srcs = '\n'.join([f'<source id=="{c["src"]}">\n{c["content"]}\n</source>' for c in context])
    srcs_short = '\n'.join([f'<source id="{c["src"]}">\n{len(c["content"])}\n</source>' for c in context])
    print(f'<sources>{srcs_short}</sources>')
    return f'<sources>{srcs}</sources>'

bing_tool = {
    'type':'function',
    'function':{
        'name':'search_and_get_text',
        # 'description':f'Searches {", ".join(bing_grounded_sites)} for a query and returns the text content of the first few results',
        'strict':True,
        'parameters':{
            'type':'object',
            'properties':{
                'query':{'type':'string'}
            },
            'required':['query'],
            'additionalProperties':False
        }
    }
}

In [ ]:
import json 
def chat(messages):
    res = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=[bing_tool]
    )
    appendix = []
    while res.choices[0].message.tool_calls:
        appendix.append(res.choices[0].message)
        for tool in res.choices[0].message.tool_calls:
            if tool.function.name == 'search_and_get_text':
                args = json.loads(tool.function.arguments)
                query = args['query']
                print(args, query)
                appendix.append({
                    'role':'tool',
                    'tool_call_id':tool.id,
                    'content': search_and_get_text(query)
                })
        messages = messages + appendix
        print(f"sending {len(messages)} messages")
        res = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )
        appendix = []
    return res.choices[0].message

In [20]:
r = chat([
    {'role':'system','content':"You are a financial advice agent. You have access to sec.gov and finra.org for regulatory information. If you don't know the answer to a user's quesiton, use the search tool that will perform seach n these websites. Always cite your answers using the id provided in sources, for example <cite id=\"1\"/>. The answers must be tailored to US customers. Do not include site: in your search query."},
    {'role':'user','content':' What are the Regulatory supervisory expectations related to Mutual Fund, ETF, UIT, and Annuity switches in retail client accounts?'},
])



{'query': 'Regulatory supervisory expectations Mutual Fund'} Regulatory supervisory expectations Mutual Fund
Searching Bing: v7.0/search?q=Regulatory supervisory expectations Mutual Fund (site:sec.gov OR site:finra.org/rules-guidance/rulebooks)
Extracted 383 characters from https://www.sec.gov/about/divisions-offices/division-investment-management
Extracted 383 characters from https://www.sec.gov/investment/laws-and-rules
Extracted 383 characters from https://www.sec.gov/investment/secg-fund-of-funds
Extracted 383 characters from https://www.sec.gov/newsroom/press-releases/2020-247
Extracted 15024 characters from https://www.finra.org/rules-guidance/rulebooks/finra-rules/3130
Extracted 383 characters from https://www.sec.gov/newsroom/press-releases/2020-302
Extracted 733 characters from https://www.finra.org/rules-guidance/rulebooks/finra-rules/3100
Extracted 4024 characters from https://www.finra.org/rules-guidance/rulebooks/finra-rules/2213
Extracted 383 characters from https://www.s

In [18]:
print(r.content)

When it comes to regulatory supervisory expectations related to mutual funds, ETFs (Exchange-Traded Funds), UITs (Unit Investment Trusts), and annuity switches in retail client accounts, here are the key points to consider:

1. **Suitability**: Advisors must ensure that each investment switch is suitable for the client based on their investment objectives, risk tolerance, financial situation, and needs. This requires a thorough understanding of the client's profile and the relative merits and risks of the products involved.

2. **Disclosure**: Advisors are expected to fully disclose the costs associated with switches, including potential sales charges, redemption fees, and tax implications. Clear communication about these costs helps clients make informed decisions.

3. **Documentation**: Detailed documentation of the rationale behind every switch is required. This documentation should include the reasons for the recommendation, benefits expected, and any potential downsides. Proper do